# CPSC-462 Group 4 Diabetes Prediction Web App


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, log_loss
from collections import defaultdict, Counter
import math
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
import pickle
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("diabetes_012_health_indicators_BRFSS2015.csv")
data.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
data.shape

(253680, 22)

### Cleaning The Data

In [ ]:
data_label = data["Diabetes_012"]
data = data.drop(["Diabetes_012"], axis = 1)
data.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
data.shape

(253680, 21)

In [ ]:
Final_Data = pd.concat([data, data_label], axis = 1)

In [ ]:
Final_Data.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_012
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0,0.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0,0.0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0,0.0
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0,0.0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0,0.0


## Spliting data 

In [ ]:
X_Train, TestData, Y_Train, TestLabels = train_test_split(data, data_label, stratify=data_label, test_size=0.2, random_state=(2022-16-2))

In [ ]:
X_Train.shape, TestData.shape, Y_Train.shape, TestLabels.shape

((202944, 21), (50736, 21), (202944,), (50736,))

In [ ]:
Train_Y = np.array(list(Y_Train))

In [ ]:
Test_Y = np.array(list(TestLabels))

### Linear Support Vector Machine(SVM) 

In [ ]:
bestvalue = 1e-06
clfs = SGDClassifier(loss = "hinge", alpha = bestvalue, class_weight = "balanced")
clfs.fit(X_Train, Train_Y)
cal_clfs = CalibratedClassifierCV(clfs, method = "sigmoid")
cal_clfs.fit(X_Train, Train_Y)

pred_train = cal_clfs.predict_proba(X_Train)
proba_train = cal_clfs.predict_proba(X_Train)
print('Best alpha value= ', bestvalue, "the train log loss =:",log_loss(Train_Y, proba_train, labels=clfs.classes_))

pred_test = cal_clfs.predict(TestData)
proba_test = cal_clfs.predict_proba(TestData)
print('Best alpha value= ', bestvalue, "the test log loss =:",log_loss(Test_Y, proba_test, labels=clfs.classes_))

Best alpha value=  1e-06 the train log loss =: 0.4092118103776222
Best alpha value=  1e-06 the test log loss =: 0.40872840440282765


In [ ]:
print ('\nSVM Train Accuracy:', metrics.accuracy_score(Train_Y, cal_clfs.predict(X_Train)))
print ('SVM Test Accuracy:', metrics.accuracy_score(Test_Y, cal_clfs.predict(TestData)))


SVM Train Accuracy: 0.8456766398612425
SVM Test Accuracy: 0.8472287921791233


In [ ]:
pickle.dump(cal_clfs,open('modelSVM.pkl','wb'))